<center><h1>IsingModelVisual</h1></center>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import trange, tqdm
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML, display
import ipywidgets as widgets
from numba import njit

In [2]:
# @njit
# def IsingModelVisual(rows, cols, J, kB, T, H, frames, speed):
#     size = int(speed * rows * cols)
#     lattice = np.random.choice(np.array([1, -1]), size=(frames, rows, cols))
#     random_rows = np.random.choice(np.arange(rows), size = (frames, size))
#     random_cols = np.random.choice(np.arange(cols), size = (frames, size))
#     probs = np.random.random(size=(frames, size))
    
#     for frame in range(frames):
#         for _, (i, j) in enumerate(zip(random_rows[frame], random_cols[frame])):
#             neighbors = lattice[frame,  (i + 1)         % rows,   j                    ] + \
#                         lattice[frame, ((i - 1) + rows) % rows,   j                    ] + \
#                         lattice[frame,   i                    ,  (j + 1)         % cols] + \
#                         lattice[frame,   i                    , ((j - 1) + cols) % cols]
#             dE = 2. * (J * neighbors - H) * lattice[frame, i, j]
#             prob = 1. if dE < 0. else np.exp(-dE / (kB * T))
#             if probs[frame, _] < prob:
#                 lattice[frame, i, j] = -lattice[frame, i, j]  
#     return lattice

In [3]:
def IsingModelVisual(rows, cols, J, kB, T, H, frames, speed):
    size = int(speed * rows * cols)
    rng = np.random.default_rng()
    lattice = rng.choice(np.array([1, -1]), size=(frames, rows, cols))
    random_rows = rng.choice(range(rows), size = (frames, size))
    random_cols = rng.choice(range(cols), size = (frames, size))
    probs = rng.random(size=(frames, size))
    
    for frame in trange(frames, desc='Sweeping Frames'):
        for _, (i, j) in enumerate(zip(random_rows[frame], random_cols[frame])):
            neighbors = lattice[frame,  (i + 1)         % rows,   j                    ] + \
                        lattice[frame, ((i - 1) + rows) % rows,   j                    ] + \
                        lattice[frame,   i                    ,  (j + 1)         % cols] + \
                        lattice[frame,   i                    , ((j - 1) + cols) % cols]
            dE = 2. * (J * neighbors - H) * lattice[frame, i, j]
            prob = 1. if dE < 0. else np.exp(-dE / (kB * T))
            if probs[frame, _] < prob:
                lattice[frame, i, j] = -lattice[frame, i, j]  
    return lattice

In [4]:
def example(rows, cols, J, kB, T, H, frames, dpi, interval, speed):
    lattice = IsingModelVisual(rows, cols, J, kB, T, H, frames, speed)
        
    figsize = np.array([cols, rows])
    figsize = 16 * 9 * figsize / np.linalg.norm(figsize)
    figure, axis = plt.subplots(figsize=figsize, dpi=dpi)
    
    print('Creating Artists... ', end='')
    artists = [ [plt.imshow(lattice[_,:,:], animated=True)] for _ in range(frames)]
    print('Done!')
    
    print('Creating Animation... ', end='')
    animation = ArtistAnimation(figure, artists, interval=interval, blit=True)
    print('Done!')
    plt.close()
    
    print('Creating Video... ', end='')
    jshtml = animation.to_jshtml()
    print('Done!')

    return display(HTML(jshtml))

In [5]:
rows     = widgets.IntSlider(  value=100, min=10,   max=1000, step=10,   continuous_update=False, description=r'$n_{\textrm{rows}}$')
cols     = widgets.IntSlider(  value=100, min=10,   max=1000, step=10,   continuous_update=False, description=r'$n_{\textrm{cols}}$')
J        = widgets.FloatSlider(value=1.,  min=0.,   max=1.,   step=0.01, continuous_update=False, description=r'$J$')
kB       = widgets.FloatSlider(value=1.,  min=0.,   max=1.,   step=0.01, continuous_update=False, description=r'$k_B$')
T        = widgets.FloatSlider(value=1.,  min=0.01, max=3.,   step=0.05, continuous_update=False, description='Temperature')
H        = widgets.FloatSlider(value=0.,  min=-10., max=10.,  step=0.05, continuous_update=False, description='B Field')
frames   = widgets.IntSlider(  value=100, min=10,   max=1000, step=10,   continuous_update=False, description=r'$n_{\textrm{frames}}$')
dpi      = widgets.IntSlider(  value=50,  min=10,   max=500,  step=25,   continuous_update=False, description=r'dpi')
interval = widgets.IntSlider(  value=200, min=100,  max=1000, step=25,   continuous_update=False, description='frame delay')
speed    = widgets.FloatSlider(value=1.,  min=1e-2, max=1.,   step=1e-2, continuous_update=False, description='speed')

In [6]:
ui_row1 = widgets.HBox([rows, cols])
ui_row2 = widgets.HBox([J, kB])
ui_row3 = widgets.HBox([T, H])
ui_row4 = widgets.HBox([frames, speed])
ui_row5 = widgets.HBox([dpi, interval])
ui = widgets.VBox([ui_row1, ui_row2, ui_row3, ui_row4, ui_row5])

In [7]:
widget = widgets.interactive_output(example, {'rows':rows, 'cols':cols, 'J':J, 'kB':kB, 'T':T, 'H':H, 'frames':frames, 'dpi':dpi, 'interval':interval, 'speed':speed})

Error in callback <function flush_figures at 0x7f404749d280> (for post_execute):


KeyboardInterrupt: 

In [6]:
display(ui, widget)

Output()